# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from datetime import datetime

# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,25.57,84,100,5.66,US,1701128333
1,1,grytviken,-54.2811,-36.5092,-0.08,96,100,10.11,GS,1701128777
2,2,bethel,41.3712,-73.4140,7.17,60,100,3.09,US,1701128348
3,3,kijang,35.2442,129.2139,6.85,55,31,4.98,KR,1701128777
4,4,ribeira grande,38.5167,-28.7000,17.10,94,86,2.24,PT,1701128336



---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    scale = 0.5,
    frame_width = 800,
    frame_height = 600,
    alpha = 0.75)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 29) &
                        (city_data_df["Max Temp"] > 18)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
86,86,east london,-33.0153,27.9116,18.82,93,0,0.39,ZA,1701128392
121,121,mount isa,-20.7333,139.5000,27.87,69,0,2.06,AU,1701128345
176,176,turbat,26.0023,63.0440,20.57,76,0,1.23,PK,1701128819
180,180,el granada,37.5027,-122.4694,18.84,38,0,1.54,US,1701128819
280,280,sao jose do rio preto,-20.8197,-49.3794,27.15,65,0,2.06,BR,1701128655



### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = pd.DataFrame({"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity})
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
86,east london,ZA,-33.0153,27.9116,93,
121,mount isa,AU,-20.7333,139.5000,69,
176,turbat,PK,26.0023,63.0440,76,
180,el granada,US,37.5027,-122.4694,38,
280,sao jose do rio preto,BR,-20.8197,-49.3794,65,
315,pacific grove,US,36.6177,-121.9166,32,
412,saint-pierre,RE,-21.3393,55.4781,64,
430,taoudenni,ML,22.6783,-3.9836,19,
454,veraval,IN,20.9000,70.3667,77,
489,bulawayo,ZW,-20.1500,28.5833,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get lat, lon from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address1 = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address1.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salalah - nearest hotel: Muscat International Hotel
mount isa - nearest hotel: Ibis Styles
dhoraji - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
pacific grove - nearest hotel: Pacific Grove Inn
gwadar - nearest hotel: Sadaf Resorts
poinciana - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
84,salalah,OM,17.0151,54.0924,73,Muscat International Hotel
100,mount isa,AU,-20.7333,139.5000,69,Ibis Styles
255,dhoraji,IN,21.7333,70.4500,78,No hotel found
256,east london,ZA,-33.0153,27.9116,93,No hotel found
342,saint-pierre,RE,-21.3393,55.4781,64,Tropic Hotel
386,pacific grove,US,36.6177,-121.9166,32,Pacific Grove Inn
459,gwadar,PK,25.1216,62.3254,67,Sadaf Resorts
478,poinciana,US,28.1403,-81.4584,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color="City",
    scale=0.5,
    frame_width = 800,
    frame_height = 600,
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)